![spam](img/sarimax.png)

# Método de Box-Jenkins 


In [1]:
import warnings
warnings.filterwarnings('ignore')

In [2]:
#%pip install pmdarima

## 1.   Exploratory Data Analysis 

<summary>
    <font size="4" color="orange"><b>1.1 Importing libraries and functions</b></font>
</summary>

In [3]:
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
%matplotlib inline
#import seaborn as sns
import plotly.graph_objects as go
import plotly.express as px

import statsmodels.tsa.stattools as sts
from statsmodels.tsa.stattools import acf,pacf,pacf_ols
from statsmodels.graphics.tsaplots import plot_acf, plot_pacf
from statsmodels.tsa.seasonal import seasonal_decompose

import scipy
from scipy import stats

#import pmdarima as pm
#from pmdarima.model_selection import train_test_split

from statsmodels.tsa.statespace.sarimax import SARIMAX
from sklearn.metrics import r2_score
import warnings
import json

# Importing PCA functions for futher processing
%run "./funciones/pcafunction.ipynb"

pd.set_option('display.max_rows', 300)

<summary>
    <font size="4" color="orange"><b>1.2 Loading CENACE database: 49 input variables </b></font>
</summary>

<img src="img/calendarsymbol.png" width="40" img align="left" />  

<font size="3" color="palevioletred"><b>Exogenous Calendar Features </b></font>

* **FECHA** (yy-mm-dd): Date

"Holiday" (0|1) indicator:

* **Lunes_Festivo**: Holiday Monday

* **Martes_PostFestivo**: Day after holiday Monday

* **Semana_Santa**: Holy Week

* **1_Mayo**: May 1

* **10_Mayo**: May 10

* **16_Sep**: September 16

* **2_Nov.**: November 2

* **Pre-Navidad_y_new_year**: Day before Christmas or New Year

* **Navidad_y_new_year**: Christmas or New Year

* **Post-Navidad_y_new_year**: Day after Christmas or New Year

<img src="img/lightsymbol.png" alt="drawing" width="25" img align="left" />  

<font size="3" color="palevioletred"><b>Endogenous Feature</b></font>

* **DEM_GCRNO_H$i$** (MW): Load energy demand in GCRNO (Gerencia de Control de Noroeste)  zone from hour $i$ to hour $i+1$ of the corresponding date, for $i=0,\dots 23$.

<img src="img/meteosymbol.png" alt="drawing" width="60" img align="left" />

<font size="3" color="palevioletred"><b>Exogenous Meteorological Features</b></font>

* **TMAX-CAB**, **TMIN-CAB**, **TMAX-HMO**, **TMIN-HMO**, **TMAX-OBR**, **TMIN-OBR**,**TMAX-LMO**, **TMIN-LMO**, **TMAX-CUL**, **TMIN-CUL** ($^\circ$C): Maximum and minimum temperature in Caborca, Hermosillo, Ciudad Obregón, Los Mochis and Culiacán, respectively.

* **PREC_HMO_mm**, **PREC_OBR_mm**, **PREC_LMO_mm**, **PREC_CUL_mm**  (mm/h): Precipitation in Hermosillo, Ciudad Obregón, Los Mochis and Culiacán, respectively.



In [4]:
# Importing load energy consumption CENACE database
url = "./inputs/Dataset GCRNO120522 DF.xlsx" #data
gcrno = pd.read_excel(url)
gcrno.columns

Index(['FECHA', 'DEM_GCRNO_H0', 'DEM_GCRNO_H1', 'DEM_GCRNO_H2', 'DEM_GCRNO_H3',
       'DEM_GCRNO_H4', 'DEM_GCRNO_H5', 'DEM_GCRNO_H6', 'DEM_GCRNO_H7',
       'DEM_GCRNO_H8', 'DEM_GCRNO_H9', 'DEM_GCRNO_H10', 'DEM_GCRNO_H11',
       'DEM_GCRNO_H12', 'DEM_GCRNO_H13', 'DEM_GCRNO_H14', 'DEM_GCRNO_H15',
       'DEM_GCRNO_H16', 'DEM_GCRNO_H17', 'DEM_GCRNO_H18', 'DEM_GCRNO_H19',
       'DEM_GCRNO_H20', 'DEM_GCRNO_H21', 'DEM_GCRNO_H22', 'DEM_GCRNO_H23',
       'TMAX-CAB', 'TMAX-HMO', 'TMAX-OBR', 'TMAX-LMO', 'TMAX-CUL', 'TMIN-CAB',
       'TMIN-HMO', 'TMIN-OBR', 'TMIN-LMO', 'TMIN-CUL', 'PREC_HMO_MM',
       'PREC_OBR_MM', 'PREC_LMO_MM', 'PREC_CUL_MM', 'LUNES_FESTIVO',
       'MARTES_POSTFESTIVO', 'SEMANA_SANTA', '1_MAYO', '10_MAYO', '16_SEP',
       '2_NOV.', 'PRE-NAVIDAD_Y_NEW_YEAR', 'NAVIDAD_Y_NEW_YEAR',
       'POST-NAVIDAD_Y_NEW_YEAR'],
      dtype='object')

<summary>
    <font size="4" color="orange"><b>1.3. Dataframe rearrangement</b></font>
</summary>

The above dataframe will be transorm in a new one with:

* *INSTANCES* (index):

    **FECHA-HORA** (Date-Hour) specified in the format yyyy-mm-dd hh:00:00
    
    
* *FEATURES*: 

    **DEMANDA** Load energy demand
    
    **DIA** (Day)
       0 Monday 
       1 Tuesday 
       2 Wednesday 
       3 Thursday 
       4 Friday 
       5 Saturday 
       6 Sunday
    
    **HORA** (Hour 0–23)
      
    **MES** (Month)
       1 January
       2 February
       3 March
       4 April
       5 May
       6 Jun
       7 July
       8 August
       9 September
       10 Octuber
       11 November
       12 December
    
    And the following characteristics with constant value with respect to the day **TMAX-CAB**, **TMIN-CAB**, **TMAX-HMO**, **TMIN-HMO**, **TMAX-OBR**, **TMIN-OBR**,**TMAX-LMO**, **TMIN-LMO**, **TMAX-CUL**, **TMIN-CUL**, **PREC_HMO_MM**, **PREC_OBR_MM**, **PREC_LMO_MM**, **PREC_CUL_MM**, **LUNES_FESTIVO**, **MARTES_POSTFESTIVO**, **SEMANA_SANTA**, **1_MAYO**, **10_MAYO**, **16_SEP**, **2_NOV.**, **PRE-NAVIDAD_Y_NEW_YEAR**, **NAVIDAD_Y_NEW_YEAR**, **POST-NAVIDAD_Y_NEW_YEAR**.

In [5]:
# Transposing hours columns from the original dataframe into rows
consumo_data = gcrno.melt(
    id_vars= ['FECHA'],
    value_vars= [f'DEM_GCRNO_H{i}' for i in range(24)],
    var_name="HORA",
    value_name="DEMANDA"
).replace(
    {f'DEM_GCRNO_H{i}': i for i in range(24)}
)
# Creating Day, Hour and Month columns
consumo_data.index = consumo_data.FECHA + pd.to_timedelta(consumo_data.HORA, unit='h')
consumo_data.sort_index(inplace=True)
consumo_data.drop(columns=['HORA'], inplace=True)
consumo_data = consumo_data.asfreq('h', method='pad')
consumo_data['FECHAHORA'] = consumo_data.index
consumo_data["DIA"] = consumo_data.index.weekday
consumo_data["HORA"] = consumo_data.index.hour
consumo_data["MES"] = consumo_data.index.month

In [6]:
# Adding columns of exogenous variables
exogenas = gcrno[['FECHA','TMAX-CAB', 'TMAX-HMO', 'TMAX-OBR', 'TMAX-LMO', 'TMAX-CUL', 'TMIN-CAB',
       'TMIN-HMO', 'TMIN-OBR', 'TMIN-LMO', 'TMIN-CUL', 'PREC_HMO_MM',
       'PREC_OBR_MM', 'PREC_LMO_MM', 'PREC_CUL_MM', 'LUNES_FESTIVO',
       'MARTES_POSTFESTIVO', 'SEMANA_SANTA', '1_MAYO', '10_MAYO', '16_SEP',
       '2_NOV.', 'PRE-NAVIDAD_Y_NEW_YEAR', 'NAVIDAD_Y_NEW_YEAR',
       'POST-NAVIDAD_Y_NEW_YEAR']]
consumo = pd.merge(consumo_data, exogenas, on='FECHA', how='left')

In [7]:
# Setting as index the DATE-HOUR
consumo.set_index("FECHAHORA", inplace=True)
consumo=consumo.asfreq('h')
consumo['FECHA-HORA'] = consumo.index

## 2.   Splitting data into training, test and validation sets


* Training: 70 %
* Validation 20 %
* Test: 10 %

For periodic retraining use all available data and separate just a validation set for the optimization process.

In [8]:
# Splitting the dataset into training, test and validation sets
df_train = consumo[consumo['FECHA-HORA'] < '2021-12-21 00:00:00']
df_test   = consumo[(consumo['FECHA-HORA'] >= '2021-03-21 00:00:00') 
                   & (consumo['FECHA-HORA'] < '2022-03-22 00:00:00')]
df_val  = consumo[(consumo['FECHA-HORA'] >= '2021-12-21 00:00:00') &
                          (consumo['FECHA-HORA'] < '2021-03-21 00:00:00') ]

## 3.    Feature engineering

The  <font size="3" color="palevioletred"><b>random features</b></font> of the training dataframe are recoded (via a principal component procedure) on the features:


* **PC1_WEATHER** 


<summary>
    <font size="4" color="orange"><b>3.1 PCA procedure </b></font>
</summary>

In [9]:
# Setting new columns from training set
pca_exogenas_train, pca_weather,scaler_model_weather= exogen_pca(df_train, 'FECHA-HORA')

# Applying training parameters to:
## Test set  

csv_data = transform_data(df_test,'FECHA-HORA',pca_weather,scaler_model_weather)
csv_data.columns

Index(['PC1_WEATHER', 'PC2_WEATHER', 'FECHA-HORA', 'DEMANDA', 'MES', 'DIA',
       'HORA', 'LUNES_FESTIVO', 'MARTES_POSTFESTIVO', 'SEMANA_SANTA', '1_MAYO',
       '10_MAYO', '16_SEP', '2_NOV.', 'PRE-NAVIDAD_Y_NEW_YEAR',
       'NAVIDAD_Y_NEW_YEAR', 'POST-NAVIDAD_Y_NEW_YEAR'],
      dtype='object')

In [11]:
csv_data = csv_data[['FECHA-HORA', 'DEMANDA', 'PC1_WEATHER', 'PC2_WEATHER','MES',
       'DIA', 'HORA', 'LUNES_FESTIVO', 'MARTES_POSTFESTIVO', 'SEMANA_SANTA',
       '1_MAYO', '10_MAYO', '16_SEP', '2_NOV.', 'PRE-NAVIDAD_Y_NEW_YEAR',
       'NAVIDAD_Y_NEW_YEAR', 'POST-NAVIDAD_Y_NEW_YEAR']]

csv_data['Fecha'] = csv_data["FECHA-HORA"]
csv_data.columns = map(str.lower, csv_data.columns)

## 4.    APPLING SARIMAX MODEL

<summary>
    <font size="4" color="orange"><b>4.1 SARIMAX function procedure </b></font>
</summary>

In [ ]:
def CSVauto_arima(csv_file,data_inicial,data_final):
    
    #Take the data of the first 16 days (24 * 16 = 384 hours) of C and perform a split, 15 days
    #(360 hours) for training and 1 day (24 hours) for testing
    
    
    data_to_autorima = csv_file.iloc[data_inicial:data_final]
    data_to_autorima.index.freq = 'h'
    result_data =[]
    csv_file.index.freq = 'h'
    Ntest = 24
    train = data_to_autorima.iloc[:-Ntest]
    test = data_to_autorima.iloc[-Ntest:]
    train_idx = csv_file.index <= train.index[-1]
    test_idx = csv_file.index > train.index[-1]
    
    #Specify which variables will make up the matrix of exogenous variables
    exogen_train = train.loc[:,'pc1_weather':'post-navidad_y_new_year']
    
    
    #Call auto_arima library, using the training data set (360 hours)
    model = pm.auto_arima(train['demanda'], d=1, seasonal=True, m=24, exog=exogen_train,
                      supress_warnings=True, trace=True)
    
    print(model)
    
    #From the results of the auto_arima model, the SARIMAX model will be configured, using
    #the training data set (360 hours) 
    sarimax = SARIMAX(train['demanda'], order=model.order, exog=exogen_train, seasonal_order=model.seasonal_order)
    
    arima_result = sarimax.fit()
    
    #Once the model is trained, the next step is to perform the forecast with the help of the
    #exogenous variables of the test data set (24 hours)
    exogen_test = test.loc[:,'pc1_weather':'post-navidad_y_new_year']
    
    prediction_result = arima_result.get_forecast(24,exog=exogen_test)
    conf_int = prediction_result.conf_int()
  
    forecast = prediction_result.predicted_mean
    csv_file.loc[test_idx, 'pronostico'] = forecast
    
    
    gerencia_mape = csv_file.iloc[data_final-24:data_final]
    
    #Calculate MAPE (1) between forecast and demand from test data.
    for i in range(0, 24):
        #MAPE DE PRONOSTICOS
        csv_file.loc[gerencia_mape.index[i], 'mape'] = mape(gerencia_mape['demanda'][i], gerencia_mape['pronostico'][i])
        
    
    return csv_file

#MAPE function
def mape(actual, pred): 
    actual, pred = np.array(actual), np.array(pred)
    return np.mean(np.abs((actual - pred) / actual)) * 100

In [ ]:
# Set data set C as the LSTM Encoder-Decoder Model test set.
# While C ̸ = ∅ : Apply CSVauto_arima function

end = 360
start = 0
result_data = []

while end < 8785:
    resultado = CSVauto_arima(csv_data, start, end)
    end += 24
    start += 24

## 5.    SARIMAX RESULT

In [ ]:
#resultado.to_csv('./outputs/sarimax_result.csv')

In [ ]:
#sarimax result dataset
resultado['fecha-hora']= pd.to_datetime(resultado['fecha-hora'])
resultado["HORA"] = resultado['fecha-hora'].dt.hour

<summary>
    <font size="4" color="orange"><b>5.1 SARIMAX daily MAPE & pearson r2 </b></font>
</summary>

In [ ]:
#daily MAPE & pearson r2
real=[]
estimada =[]

resultado = pd.DataFrame(columns=['fecha_hora','real','estimado','mape', 'r2'])

for i, row in resultado.iterrows():

    n = row['HORA']
    
    real.append(row['demanda'])
    estimada.append(row['pronostico'])
    
   
    if n == 23:
        df_all_results_sarimax = df_all_results_sarimax.append(pd.DataFrame({
                                           "fecha_hora": row['fecha'],
                                           "real":row['demanda'],
                                           "estimado":row['pronostico'],
                                           "mape": (mape(real, estimada))*100, 
                                           "r2": stats.pearsonr(real, estimada)[0]}, 
                                            index=[0]), ignore_index=True)
        n=0
        
        real=[]
        estimada =[]
    n+=1

In [ ]:
#ploting time series sarimax mape
fig = px.line(df_all_results_sarimax,x='fecha_hora', y='mape' ,title="SARIMAX MAPE Test data: from 2021-03-21 to 2022-03-21")
fig.add_hline(y=5, line_dash="dot", line_color="red"),
fig.show()

In [ ]:
#ploting time series sarimax pearson r2
fig = px.line(df_all_results_sarimax,x='fecha_hora', y='r2' ,title="SARIMAX R2 Test data: from 2021-03-21 to 2022-03-21")
fig.add_hline(y=0.95, line_dash="dot", line_color="red"),
fig.show()

In [ ]:
#sarimay daily mape violin plot
fig = px.violin(df_all_results_sarimax, y="mape", box=True, # draw box plot inside the violin
                points='all', # can be 'outliers', or False
               )
fig.show()

<summary>
    <font size="4" color="orange"><b>5.2 SARIMAX Result spliting 4 Seasons </b></font>
</summary>

In [ ]:
#spliting on 4 seasons
sarimax_summer = df_all_results_sarimax[(df_all_results_sarimax['fecha_hora'] >= '2021-06-21') 
                   & (df_all_results_sarimax['fecha_hora'] < '2021-09-23')]

sarimax_autumn = df_all_results_sarimax[(df_all_results_sarimax['fecha_hora'] >= '2021-09-23') 
                   & (df_all_results_sarimax['fecha_hora'] < '2021-12-21')]

sarimax_winter = df_all_results_sarimax[(df_all_results_sarimax['fecha_hora'] >= '2021-12-21') 
                   & (df_all_results_sarimax['fecha_hora'] < '2022-03-21')]

sarimax_spring = df_all_results_sarimax[(df_all_results_sarimax['fecha_hora'] >= '2021-03-21') 
                   & (df_all_results_sarimax['fecha_hora'] < '2021-06-21')]

In [ ]:
#4 seasons violin plots 
fig = go.Figure()

fig.add_trace(go.Violin(y=sarimax_spring['mape'],
                        name='SPRING SARIMAX MAPE',
                        box_visible=True,
                        meanline_visible=True,
                        fillcolor="rgba(158, 202, 57,0.5)", 
                        line=dict(color="#9eca39")))

fig.add_trace(go.Violin(y=sarimax_summer['mape'],
                        name='SUMMER SARIMAX MAPE',
                        box_visible=True,
                        meanline_visible=True,
                        fillcolor="rgba(249, 190, 4,0.5)", 
                        line=dict(color="#F9BE04")))

fig.add_trace(go.Violin(y=sarimax_autumn['mape'],
                        name='AUTUMN SARIMAX MAPE',
                        box_visible=True,
                        meanline_visible=True,
                        fillcolor="rgba(198, 111, 66,0.5)", 
                        line=dict(color="#C66F42")))

fig.add_trace(go.Violin(y=sarimax_winter['mape'],
                        name='WINTER SARIMAX MAPE',
                        box_visible=True,
                        meanline_visible=True,
                        fillcolor="rgba(112, 163, 187,0.5)", 
                        line=dict(color="#70A3BB")))



fig.update_layout(title_text=" SARIMAX MAPE 4 SEASONS", height=600) 
fig.show()

In [ ]:
#4 seasons MAPE time series plots 

from plotly.subplots import make_subplots
fig = make_subplots(rows=4, cols=1)


fig.add_trace(go.Scatter(x=sarimax_spring['fecha_hora'], y=sarimax_spring['mape'],
            mode='lines+markers',
            name='SARIMAX MAPE Test data: Spring', 
            marker = { 'color':'#9eca39'}),
            row=1, col=1)
fig.add_hline(y=5, line_dash="dot", line_color="red"),
fig.add_trace(go.Scatter(x=sarimax_summer['fecha_hora'], y=sarimax_summer['mape'],
            mode='lines+markers',
            name='SARIMAX MAPE Test data: Summer', 
            marker = { 'color':'#F9BE04'}),
            row=2, col=1)
fig.add_hline(y=5, line_dash="dot", line_color="red"),
fig.add_trace(go.Scatter(x=sarimax_autumn['fecha_hora'], y=sarimax_autumn['mape'],
            mode='lines+markers',
            name='SARIMAX MAPE Test data: Autumn', 
            marker = { 'color':'#C66F42'}),
            row=3, col=1)
fig.add_hline(y=5, line_dash="dot", line_color="red"),
fig.add_trace(go.Scatter(x=sarimax_winter['fecha_hora'], y=sarimax_winter['mape'],
            mode='lines+markers',
            name='SARIMAX MAPE Test data: Winter', 
            marker = { 'color':'#70A3BB'}),
            row=4, col=1)
fig.add_hline(y=5, line_dash="dot", line_color="red"),

fig.update_layout(yaxis = dict(range=[0, 16]))                                         
fig.update_layout(title_text="SARIMAX MAPE Test data: 4 Seasons", height=1200)                                         
fig.show()

In [ ]:
#4 seasons pearson r2 time series plots

fig = make_subplots(rows=4, cols=1)


fig.add_trace(go.Scatter(x=sarimax_spring['fecha_hora'], y=sarimax_spring['r2'],
            mode='lines+markers',
            name='SARIMAX R2 Test data: Spring', 
            marker = { 'color':'#9eca39'}),
            row=1, col=1)
fig.add_hline(y=0.95, line_dash="dot", line_color="red"),
fig.add_trace(go.Scatter(x=sarimax_summer['fecha_hora'], y=sarimax_summer['r2'],
            mode='lines+markers',
            name='SARIMAX R2 Test data: Summer', 
            marker = { 'color':'#F9BE04'}),
            row=2, col=1)
fig.add_hline(y=0.95, line_dash="dot", line_color="red"),
fig.add_trace(go.Scatter(x=sarimax_autumn['fecha_hora'], y=sarimax_autumn['r2'],
            mode='lines+markers',
            name='SARIMAX R2 Test data: Autumn', 
            marker = { 'color':'#C66F42'}),
            row=3, col=1)
fig.add_hline(y=0.95, line_dash="dot", line_color="red"),
fig.add_trace(go.Scatter(x=sarimax_winter['fecha_hora'], y=sarimax_winter['r2'],
            mode='lines+markers',
            name='SARIMAX R2 Test data: Winter', 
            marker = { 'color':'#70A3BB'}),
            row=4, col=1)
fig.add_hline(y=0.95, line_dash="dot", line_color="red"),

fig.update_layout(yaxis = dict(range=[0, 1.1]))                                         
fig.update_layout(title_text="SARIMAX R2 Test data: 4 Seasons", height=1200)                                         
fig.show()

from IPython import display
display.Image("https://mcd.unison.mx/wp-content/themes/awaken/img/logo_mcd.png", embed = True)

<summary>
    <font size="4" color="gray"> Maestría en Ciencia de Datos | Universidad de Sonora </font>
</summary>
<font size="1" color="gray"> Blvd. Luis Encinas y Rosales s/n Col. Centro. Edificio 3K1 planta baja C.P. 83000, Hermosillo, Sonora, México </font>
<font size="1" color="gray"> mcd@unison.mx </font>
<font size="1" color="gray"> Tel: +52 (662) 259 2155  </font>